In [ ]:
import h5py, sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
sys.path.append('/mnt/cbis/home/benedikt/.local/dragonfly/utils/py_src')

In [ ]:
import writeemc
import detector
import reademc

In [ ]:
path_to_data = "../../data/sparse/" # scratch/sparse on Maxwell
path_to_aux = "../../data/aux/" # scratch/benedikt/aux on Maxwell
path_to_geometry = "../../geometry/"
path_to_recons = "../../data/recons/"

## Load EMC reconstruction

In [ ]:
rr = 2

In [ ]:
emc_folder = path_to_recons + "melv_0001/"
emc_output = emc_folder + "data/r%d/output_060.h5" %rr
emc_photons = emc_folder + 'photons.txt'
emc_detector = path_to_recons + "det/det_2145_lowq1.h5"

In [ ]:
with h5py.File(emc_output,'r') as f:
    occupancies = f['occupancies'][:]
    likelihood = f['likelihood'][:]
    orientations = f['orientations'][:]
    images = f['intens'][:]

## Determine modes

In [ ]:
nr_rot = 180
nr_static = 0
nr_modes = occupancies.shape[1] - nr_static
modes = orientations % (nr_modes)
rotind = orientations // (nr_modes)
modes[rotind >= nr_rot] = orientations[rotind >= nr_rot] - nr_modes * (nr_rot - 1)
blacklisted = (occupancies == 0).all(axis=1)
ndata = len(modes) - blacklisted.sum()

## Inspect 2D classes

In [ ]:
N = occupancies.shape[1]
ncols = 10
nrows = N//ncols
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4,nrows*4), dpi=200)
for i in range(N):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
    nclass = ((modes==i) & (~blacklisted)).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[i], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %i, color='r')
plt.savefig("../../plots/2dclasses.png", bbox_inches='tight')
plt.show()

## Inspect single classes

In [ ]:
good = 0,2,8,24,29,36,39,40,49,55
N = len(good)
ncols = 5
nrows = int(np.ceil(N/ncols))
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*4,nrows*4), dpi=100)
for i in range(ncols*nrows):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
for i in range(N):
    c,r = i//ncols, i%ncols
    nclass = ((modes==good[i]) & (~blacklisted)).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[good[i]], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %good[i], color='r')
plt.show()

## Inspect maybe classes

In [ ]:
maybe = 4,19,41,50
N = len(maybe)
ncols = 2
nrows = int(np.ceil(N/ncols))
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*4,nrows*4), dpi=100)
for i in range(ncols*nrows):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
for i in range(N):
    c,r = i//ncols, i%ncols
    nclass = ((modes==maybe[i]) & (~blacklisted)).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[maybe[i]], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %maybe[i], color='r')
plt.show()

## Classes to be removed/kept

In [ ]:
keep = list(good)
remove = list(range(nr_modes))
for k in keep: remove.remove(k)

## Inspect classes to remove

In [ ]:
N = len(remove)
ncols = 5
nrows = int(np.ceil(N/ncols))
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*4,nrows*4), dpi=100)
removetotal = 0
for i in range(ncols*nrows):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
for i in range(N):
    c,r = i//ncols, i%ncols
    nclass = ((modes==remove[i]) & (~blacklisted)).sum()
    removetotal += nclass
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[remove[i]], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %remove[i], color='r')
plt.show()

In [ ]:
print("Keeping %d frames for 3D [%.2f%%]" %(ndata-removetotal, (ndata-removetotal)/ndata*100))

## Write selection into blacklist file

In [ ]:
blacklist = np.zeros(occupancies.shape[0], dtype=int)
for m in remove:
    blacklist[np.where(modes == m)[0]] = 1
ndata = blacklist.shape[0] - blacklist.sum()

In [ ]:
np.savetxt(emc_folder + "blacklist_%d_%d.dat" %(rr,ndata), blacklist, fmt='%d')